In [ ]:
!nvidia-smi


# Deep Learning for Natural Language Processing

Natural Language Processing (NLP) covers machine learning techniques dealing with text and includes

* classification
  * sentiment analyis (is this tweet a Pro-Biden or Anti-Biden one)
  * stylometrics (was this typed suicide note really from the deceased or did the murderer write it [1](https://www.rosette.com/case-studies/alias/), [2](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3107011/)
  * general classification (out of 40 topics, which is this article about)
* question answering (building systems that can answer questions -- *What is the best treatment for hemangiosarcoma in dogs?*)
* machine translation
* speech recognition

among many others. Deep Learning has led to tremendous improvements in all these areas of NLP. 

In this notebook, we are going to examine classification systems for textual information.





## Analyzing and Classifying Text



<img src="https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/tiles.jpg" width="500"/>

So far we have been dealing with **structured data**. Structured data is ... well ... structured. This means that an instance of our data has nice attributes that can be represented in a DataFrame or a table:

make | mpg | cylinders | HP | 0-60 |
---- | :---: | :---: | :---: | :---: |
Fiat | 38 | 4 | 157   | 6.9 
Ford F150 | 19 | 6 | 386 | 6.3 
Mazda 3 | 37 | 4 | 155 |  7.5 
Ford Escape | 27 | 4 | 245 | 7.1 
Kia Soul | 31 | 4 | 164 | 8.5 

The majority of data in the world is **unstructured**. Take text for example. Suppose I have a corpus of twitter posts from former president Donald Trump and the Dalai Lama and my goal is to create a classifier that takes a tweet and tells me if it was produced by Trump or the Dalai Lama:

*The purpose of education is to build a happier society, we need a more holistic approach that promotes the practice of love and compassion.*

*How low has President Obama gone to tapp my phones during the very sacred election*

We might consider  the columns of a table to be things like *first word of the tweet*, *second word of the tweet* and so on:


id | word 1 | word 2 | word 3 | word 4 |word 5 |word 6 | ... |
---- | :---: | :---: | :---: | :---: | :---: |:---: |:---: |
1 | The | purpose | of   | education |is | to | ...
2 | How | low | has |President | Obama | gone | ...

So we would be counting how many times the word *President* occurred as the fourth word of a tweet. **But that would be the wrong way to go**. First, the deep learning models we have developed so far require input of a specific length. For example, we resized our dog and cat images to a uniform 150x150 and because each image was a uniform size we could specify the input shape of our network to be ....

```
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
```

But what should the input length be for tweets? Sometimes there are short, one-word, tweets like *nice*. The average length of a tweet is 30 characters so something like the six word *Today, Toaster is 10 years old.*  And the limit of course is 280 characters so...

> Like anything else, life has a beginning and in due course must end. In between those two events the important goal should be to live meaningfully, not to create trouble for others. If we can do that, when the end comes, we can go feeling at peace.

That's 47 words so maybe we can limit our input to 50 words and for shorter tweets we can pad them with blank words.  But there is another possibility ...

### Bag of Words (bow)

A more common way to represent text is to treat the text as an unordered set of words, which is called the **bag of words** approach. 


<img src="https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/BagofWords.jpg" width="350"/>

With the bag of words approach we count word occurrences and the features (what we might think of as columns) are the unique words. For example, we have a collection of Trump and Dalai Lama tweets and indicate whether the word occurred in the tweet or not. So we might get something like:

id | a | the | compassion | love |sad |fake | rigged | ... |
---- | :---: | :---: | :---: | :---: | :---: |:---: |:---: | :---:
Trump_1 | 1 | 0 | 0   | 0 |1 | 1 | 1 |...
Trump_2 | 1 | 1 | 0   | 0 | 0 | 1 | 1 |...
DalaiLama_1 | 1 | 1 | 1 |1 | 0 | 0 | 0 | ...

So, for example, in DalaiLama_1, in the text there was

* an occurrence of *a*
* an occurrence of *the*
* an occurrence of *compassion*
* an occurrence of *love*

We don't know what order the words occurred in, we just know what words occurred in the tweet. This is the bag-of-words method. 

Instead of short text snippets like tweets, let's say we are analyzing speeches of Trump and the Dalai Lama. Maybe then we will count how many times they used each word. So something like:

id | a | the | compassion | love |sad |fake | rigged | ... |
---- | :---: | :---: | :---: | :---: | :---: |:---: |:---: | :---:
Trump_1 | 52 | 25 | 0   | 0 |21 | 82 | 19 |...
Trump_2 | 30 | 35 | 0   | 0 | 5 | 20 | 31 |...
DalaiLama_1 | 60 | 271 | 27 |63 | 12 | 0 | 0 | ...

Of course we are still faced with how many columns to make, representing the vocabulary size. We could limit it to the 10,000 most common words, for example.

Once we have the text in this format we can use the standard deep learning classification techniques we used before.


Converting **unstructured** text to something **structured** is a multistep process. Let's learn the bits before putting it together. And we will start with the last step first-- creating the bag of words.

### Import Keras ...



In [ ]:
import keras
keras.__version__

### Some sample data

In [ ]:
trump1 = "How low has President Obama gone to tapp my phones during the very sacred election process. This is Nixon/Watergate. Obama bad (or sick) guy! Sad"
trump2 = "Our wonderful new Healthcare Bill is now out for review and negotiation. ObamaCare is a complete and total disaster - is imploding fast! Sad"
trump3 = "Don't let the FAKE NEWS tell you that there is big infighting in the Trump Admin. We are getting along great, and getting major things done!"
trump4 = "Russia talk is FAKE NEWS put out by the Dems, and played up by the media, in order to mask the big election defeat and the illegal leaks! Sad"
dalaiLama1 = "The purpose of education is to build a happier society, we need a more holistic approach that promotes the practice of love and compassion."
dalaiLama2 = "Be a kind and compassionate person. This is the inner beauty that is a key factor to making a better world."
dalaiLama3 = "If our goal is a happier, more peaceful world in the future, only education will bring change."
dalaiLama4 = "Love and compassion are important, because they strengthen us. This is a source of hope"
tinyCorpus = [trump1, trump2, trump3, trump4, dalaiLama1, dalaiLama2, dalaiLama3, dalaiLama4]
tinyCorpus

### Create the bag of words

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=200)
tokenizer.fit_on_texts(tinyCorpus)

# Directly get the one-hot binary representations.
# Note that other vectorization modes than one-hot encoding are supported!
one_hot_results = tokenizer.texts_to_matrix(tinyCorpus, mode='binary')
# let's look at an example of an encoding ...
print(one_hot_results[0])


# This is how you can recover the word index that was computed
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

That was pretty easy. And now we have the texts in a form we can use for deep learning.

Instead of the binary choice (a 1 if the word is present and a 0 if not) -- `mode='binary'` we can count how many occurrences of each word there were in the text by using `mode='count'`:


In [ ]:
one_hot_results = tokenizer.texts_to_matrix(tinyCorpus, mode='count')
# let's look at an example of an encoding ...
print(one_hot_results[1])

So in the first tweet:

> 'Our wonderful new Healthcare Bill is now out for review and negotiation. ObamaCare is a complete and total disaster - is imploding fast! Sad',

There were 3 occurrences of the word *is*, 2 of *and*, and so on as indicated in the first row above.

How do we know what columns are associated with which words? We can use the word_index.



In [ ]:
# tokenizer.word_index is a python dictionary containing the word as a key and the column as its value
[(k, v) for k, v in sorted(tokenizer.word_index.items(), key=lambda item: item[1])]

So, *the, is, and, a, to* are the words represented by the first five columns.


# TF-IDF representation

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/bigTFIDF.png)
So far we looked at 

* a binary bag-of-words (whether or not the word was present in the text).
* a raw count bag-of-words (counting how many occurrences of each word)

There are several other approaches
We could represent a document as a bag of words and their probabilities (`mode="freq"`). For example, in *Tom Sawyer* 4.6% of the words are *the* and 0.95% are *Tom*. But the word *the* probably occurs in most novels with that frequency. So in some sense the word *the* is uninteresting. On the other hand *Tom* probably occurs much more frequently in *Tom Sawyer* than it does in *Moby Dick* and, in that way, it is a more interesting word. One way to discount words that occur evenly throughout our document collection is to use TF-IDF.  

* TF: Term Frequency - each word uprated by how often the word occurs in the document.
* IDF Inverse Document Frequency - how often the word appears in the entire corpus



> TF-IDF was first proposed by Karen Sparck Jones as a heuristic--not having any theoretical foundation. Researchers since then have tried to justify this metric by relating it to probablistic theories and information theories. But these attempts have been problematic. Nevertheless, as we will see, this heuristic works quite well. 

The formula is

### $$ tfidf(t, d) = tf(t,d) \times idf(t) $$

where *t* is the term (the word) and *d* is the document.

To explain this I will use some made up data--the word counts of 5 emails (and for the sake of later computations let's assume that each email is 100 words long):

id | the | sad | compassion |  
----: | :---: | :---: | :---:
1 | 3 | 0 | 1 
2 | 3 | 0 | 0 
3 | 4 | 0 | 0 
4 | 3 | 2 | 0 
5 | 3 | 0 | 2


The intuition is this. Even though the word *the* occurs frequently in each email, it is unlikely to help us classify email because it occurs in **every** email. The words *sad* and *compassion* are more interesting as they don't occur uniformly in our collection. 

#### TF

The TF part of TF-IDF refers to how often the word occurs in the document. There are a number of ways to define TF. The simplist is to use the raw count.  So for example, the TF of *the* in document 1 is 3 (the word *the* occurred 3 times in document 1). One problem with this approach is that the raw count is influenced by the length of the document. So if you in your 1,000 word essay on Tom Sachs use 50 occurrences of *the* and Jane in her 90,000 word Zen van life mystery novel use 4,5000 occurrences of *the*, it doesn't mean that Jane is a bigger fan of *the* than you are. Even though there is that disparity in the raw counts it is not a characteristic that will help us distinguish texts about Zen from those about Tom Sachs.  In both the 1,000 word essay and the 90,000 word book about 5% of the words are *the*. A popular measure of TF is to divide the number of occurrences of a word by the total words in the document. So the TF of the word *the* in both your 1,000 word essay and my novel would be .05




#### IDF

IDF is defined as:

### $$ idf(t)=\log\frac{1+n_d}{1+df(d,t)}+ 1 $$

$n_d$ is the total number of documents and $df(d,t)$ is how many documents the term *t* occurred in. 

So:

### $$ idf(the)=\log\frac{1+5}{1+5}+ 1 =  1.5 $$

### $$ idf(compassion)=\log\frac{1+5}{1+2}+ 1 = \log{2} + 1 =  2 $$

So, *the* in document 1 has a tf-idf of $.03 \times 1.5 = 0.045$ and *compassion* has a tf-idf of $.01 \times \ 2 = 0.02$

This is a fairly important concept to understand. I was asked about tf-idf in my oral exam to become a certified instructor at the Deep Learning Institute and fortunately I knew about it. 

It is also important to know that while it works well as a heuristic and has been around since the 70s, there really is no theoretical foundation to it. 

With all that as background, it is easy to convert a document collection into an array of TFIDF values:

In [ ]:
one_hot_results = tokenizer.texts_to_matrix(tinyCorpus, mode='tfidf')
# let's look at an example of an encoding ...
print(one_hot_results[1])

Again, once we have this representation we can use the deep learning methods we already used.


## An initial example - IMDB

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/imdb.png)

The Internet Movie DataBase consists of 50,000 movie reviews and contains an equal number of positive and negative reviews. The task of identifying the affect of a text (whether it is postive or negative, or how strongly someone feels about the topic) is called **sentiment analysis**. 

### Load the data

In [ ]:
!wget http://zacharski.org/files/courses/cs419/imdb.zip
!unzip imdb.zip

In [ ]:
!ls -l

The zip file only contained one file so we could have read it directly.  Since we already unzipped it let's use the unzipped version.

In [ ]:
import pandas as pd
data = pd.read_csv('imdb.csv')
data

Now let's separate the texts from the labels. Also note that the labels are the strings *positive* and *negative* so let's convert those to 1 and 0.

In [ ]:
data_text = data.review
data_label =data.sentiment
data_label =  data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
data_label 

### TFIDF
Now we are going to convert the text represented as strings to a tfidf representation. 

Let's use the 5,000 most common words in the documents (`Tokenizer(num_words=5000)`)

This will take a bit of time!

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data_text)

# Directly get the one-hot binary representations.
# Note that other vectorization modes than one-hot encoding are supported!
one_hot_results = tokenizer.texts_to_matrix(data_text, mode='tfidf')
# let's look at an example of an encoding ...
print(one_hot_results[0])

###  Divide into training and test sets

In [ ]:
from sklearn.model_selection import train_test_split
imdb_train_text, imdb_test_text, imdb_train_labels, imdb_test_labels = train_test_split(one_hot_results, data_label, test_size = 0.2, random_state=42)
imdb_test_labels


### Build a deep learning model
Let's go with a basic, no frills, model:

In [ ]:
import keras
keras.__version__

In our tokenizer we specified a vocabulary size of 5,000 words, so that is our `input_shape`. We are trying to predict a binary 1,0 classification so we need 

```
network.add(layers.Dense(1, activation='sigmoid'))
```
at the very end of our network. 

We would like a network with

1. a dense layer with 512 nodes and input shape (5000,)
2. a dense layer of 256
3. A dense layer of 128
4. A dense layer (the output layer) of 1 with the sigmoid activation function.


In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(5000,)))
network.add(layers.Dense(256, activation='relu'))
network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dense(1, activation='sigmoid'))

Again, we are predicting a binary 1,0 classification (was it a positive review or not) so we will use `binary_crossentropy` as our loss function

In [ ]:
from keras import optimizers
network.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4),
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
network.summary()

### fitting to the data
Now it is time to fit the network to the data. Let's use 20% of the data for validation and run for 30 epochs.

In [ ]:
history = network.fit(
      imdb_train_text, imdb_train_labels,
      steps_per_epoch=100,
      epochs=30,
      validation_split=0.2,
      validation_steps=50)

### Our accuracy and loss
Let's plot out both the training and validation accuracy and loss.

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

What is your interpretation of the plots?

.

.

.


Okay, that definitely looks like overfitting. We will examine ways of ameliorating overfitting shortly.  

Let's see how our network performs on the test data

In [ ]:
scoreSeg = network.evaluate(imdb_test_text, imdb_test_labels)
print("Accuracy: ", scoreSeg[1])

Not bad for our first attempt at text classification!!!

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)


# <font color='#EE4C2C'>Climate Change ...</font> 



![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/factfake.jpeg)

On Twitter there are people who deny climate change:

> I don't know about you guys, but I think climate change is -- as Lord Monckton said -- bullsh*t

and people who believe it is real:

> Millennials, and Gen z, and all these folks that come after us, are looking up and we’re like ‘the world will end in 12 years if we don’t address climate change, and your biggest issue is how are we gonna pay for it?

We are going to investigate the sentiment of tweets from the Twitter Climate Change Sentiment Dataset compiled by Edward Qian. The dataset consists of 43,943 tweets. Instead of a binary label (a positive sentiment on climate change or a negative, there are four possible labels:

Label | Description
:--- | :---- 
News | the tweet links to factual news about climate change
Pro | the tweet supports the belief of man-made climate change
Neutral |  the tweet neither supports nor refutes the belief of man-made climate change
Anti | the tweet does not believe in man-made climate change


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>1. Load the data.</font> 

The file is 

https://raw.githubusercontent.com/zacharski/ml-class/master/data/climateSentiment.csv.zip

You will need:

* to load the file
* convert the text of the tweet to a tf-idf representation. We will start with using the 5,000 most common words
* convert the labels
* divide into training and testing.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>2. Create a deep learning densely connected network.</font> 

You can decide how many layers and how many nodes per layer. Keep in mind that for the imdb task the last layer was:

```
network.add(layers.Dense(1, activation='sigmoid'))
```

The `1` was selected because we only had a binary choice (positive or negative). `sigmoid` was also selected because we had a binary choice.  This will not be the same for this task. In the imdb example, we used the binary_crossentropy loss function because, again, we only had a binary choice.

Create the network, compile it, and fit it to the data. 

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>2. Plot the accuracy and loss for both the training and validation sets.</font> 

Also, state in a few sentences what you see in the plots.

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>4. Accuracy on the test data</font> 

What is the accuracy on the test data?

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/PyDivideTwo.png)
## <font color='#EE4C2C'>5. Can you do better than the baseline accuracy? +5-10 xp</font> 
Can you create a network that has better accuracy than that shown in #4 above?

You can change:

* the number of layers, 
* the number of nodes in each layer
* change the `num_words` used in the tokenizer
* add one or more dropout layers
